In [1]:
import pandas as pd
import numpy as np
import polars as pl
import datetime

pd.set_option("display.max_columns", None)

In [2]:
### Target col
[
    "hospcode",
    "hosname",
    "name_th",
    "county",
    "changwatname",
    "ampurname",
    "tambonname",
    "sex",
    "birth",
    "age",
    "nation",
    "typearea",
    "diagcode",
    "diagtype",
    "date_serv",
    "chiefcomp",
    "occupation",
    "diagname",
    "typediag",
]
### Current_col
current_col = [
    "opd.hospcode",
    "c.hosname",
    "opd.pid",
    "opd.seq",
    "p.cid",
    "p.sex",
    "p.birth",
    "age",
    "p.nation",
    "p.typearea",
    "opd.diagcode",
    "opd.diagtype",
    "opd.date_serv",
    "p.occupation_new",
    "occ.occupation_new",
    "p.occupation_old",
    "old.occupation",
    "c.provcode",
    "ch.changwatname",
    "c.distcode",
    "ca.ampurname",
    "c.subdistcode",
    "ct.tambonname",
    "s.instype",
    "s.typeout",
    "s.chiefcomp",
]

In [3]:
occ = pl.read_excel(
    "./utilities/occupations.xlsx", schema_overrides={"code": pl.String}
)

# hoscode = pl.read_csv("./utilities/hospcode.csv", dtypes={"hospcode": str}).select(
#     [
#         "hospcode",
#         "name_th",
#         pl.col("changwatname").alias("province"),
#         pl.col("ampurname").alias("district"),
#         pl.col("tambonname").alias("subdistrict"),
#     ]

hoscode = pl.read_csv("./utilities/filled_hos.csv", dtypes={"5_number": str}).select(
    ["5_number", "hosname", "odpc", "province", "district", "subdistrict"]
)

total_pop = (
    pl.read_csv("./utilities/total_pop.csv", dtypes={"year": pl.String})
    .select(["year", "total_pop", "DistrictThaiShort", "ProvinceThai"])
    .group_by(by=["year", "DistrictThaiShort", "ProvinceThai"])
    .agg(pl.col("total_pop").sum())
)


icd10 = pl.read_csv("./utilities/icd10.csv", dtypes={"code": pl.String}).unique()


nation = pl.read_csv("./utilities/nation.csv", dtypes={"code": pl.String}).vstack(
    pl.DataFrame({"code": ["99", "57", "48"], "nation": ["ไทย", "กัมพูชา", "เมียนมา"]})
)


odpc = (
    pl.read_excel("./utilities/Template_เขตสุขภาพ_ภูมิภาค.xlsx")
    .with_columns(pl.col("Regional Health").str.strip_prefix("เขตสุขภาพที่ "))
    .select(["province", pl.col("Regional Health").alias("odpc")])
)

C:\Users\brown\AppData\Local\Temp\ipykernel_14308\2184070839.py:14: DeprecationWarning: The argument `dtypes` for `read_csv` is deprecated. It has been renamed to `schema_overrides`.
  hoscode = pl.read_csv("./utilities/filled_hos.csv", dtypes={"5_number": str}).select(
C:\Users\brown\AppData\Local\Temp\ipykernel_14308\2184070839.py:19: DeprecationWarning: The argument `dtypes` for `read_csv` is deprecated. It has been renamed to `schema_overrides`.
  pl.read_csv("./utilities/total_pop.csv", dtypes={"year": pl.String})
C:\Users\brown\AppData\Local\Temp\ipykernel_14308\2184070839.py:21: DeprecationWarning: named `by` param is deprecated; use positional `*args` instead.
  .group_by(by=["year", "DistrictThaiShort", "ProvinceThai"])
C:\Users\brown\AppData\Local\Temp\ipykernel_14308\2184070839.py:26: DeprecationWarning: The argument `dtypes` for `read_csv` is deprecated. It has been renamed to `schema_overrides`.
  icd10 = pl.read_csv("./utilities/icd10.csv", dtypes={"code": pl.String}).uni

In [4]:
odpc

province,odpc
str,str
"""แม่ฮ่องสอน""","""1"""
"""แพร่""","""1"""
"""เลย""","""8"""
"""เพชรบูรณ์""","""2"""
"""เพชรบุรี""","""5"""
…,…
"""กำแพงเพชร""","""3"""
"""กาฬสินธุ์""","""7"""
"""กาญจนบุรี""","""5"""


In [5]:
hoscode

5_number,hosname,odpc,province,district,subdistrict
str,str,str,str,str,str
"""32045""","""คลินิกรักษ์ฟัน""","""เขตสุขภาพที่ 9""","""นครราชสีมา""","""เมืองนครราชสีมา""","""จอหอ"""
"""32047""","""คลินิกนายแพทย์คธาวุฒิ""","""เขตสุขภาพที่ 9""","""นครราชสีมา""","""ขามทะเลสอ""","""ขามทะเลสอ"""
"""32051""","""คลินิกนายแพทย์เรืองศักดิ์""","""เขตสุขภาพที่ 9""","""นครราชสีมา""","""ครบุรี""","""แชะ"""
"""32053""","""คลินิกทีปวิทย์การแพทย์""","""เขตสุขภาพที่ 9""","""นครราชสีมา""","""ครบุรี""","""บ้านใหม่"""
"""32065""","""คลินิกแพทย์นิพนธ์""","""เขตสุขภาพที่ 9""","""นครราชสีมา""","""โนนไทย""","""โนนไทย"""
…,…,…,…,…,…
"""49992""","""อุไรวรรณคลินิกการพยาบาลและการผ…","""เขตสุขภาพที่ 10""","""อุบลราชธานี""","""นาจะหลวย""","""โนนสมบูรณ์"""
"""49993""","""คลินิกปิยะนุชการพยาบาลและผดุงค…","""เขตสุขภาพที่ 7""","""ขอนแก่น""","""ชุมแพ""","""หนองเขียด"""
"""49994""","""กัลยกรคลินิกการพยาบาลและการผดุ…","""เขตสุขภาพที่ 8""","""สกลนคร""","""พังโคน""","""ไฮหย่อง"""


In [6]:
filtered_hos = hoscode.filter(hoscode["5_number"] == "00935")
filtered_hos

5_number,hosname,odpc,province,district,subdistrict
str,str,str,str,str,str
"""00935""","""สถานีอนามัยเฉลิมพระเกียรติ 60 …","""เขตสุขภาพที่ 6""","""สมุทรปราการ""","""เมืองสมุทรปราการ""","""บางเมือง"""


In [183]:
# data = (
#     pl.read_csv(
#         "I21_I22_I24_OPD_Y2024M01-05_20240620.csv",
#         has_header=False,
#         new_columns=current_col,
#         dtypes={
#             "column_1": str,
#             "column_4": str,
#             "column_24": str,
#             "column_3": str,
#             "column_14": str,
#             "column_12": str,
#             "column_25": str,
#             "column_9": str,
#         },
#     )
#     .drop_nulls(subset=["p.cid"])
#     .drop(
#         columns=[
#             "c.hosname",
#             "c.provcode",
#             "ch.changwatname",
#             "c.distcode",
#             "ca.ampurname",
#             "c.subdistcode",
#             "ct.tambonname",
#         ]
#     )
#     .filter(
#         (~pl.col("s.chiefcomp").str.contains("รับยา|ตามนัด|F/U|ต่อยา|มาตามนัด"))
#         | (
#             (pl.col("s.chiefcomp").str.contains("รับยา|ตามนัด|F/U|ต่อยา"))
#             & pl.col("s.chiefcomp").str.contains("มีอาการ")
#         )
#     )
#     .with_columns(
#         zfill=pl.col("opd.hospcode").cast(pl.String).str.zfill(5).alias("opd.hospcode")
#     )
# )

# # แสดงผลลัพธ์
# print(data)

In [8]:
data = (
    pl.read_csv(
        "I21_I22_I24_OPD_Y2024M01-05_20240620.csv",
        has_header=False,
        new_columns=current_col,
        dtypes={
            "column_1": str,
            "column_4": str,
            "column_24": str,
            "column_3": str,
            "column_14": str,
            "column_12": str,
            "column_25": str,
            "column_9": str,
        },
    )
    .drop_nulls(subset=["p.cid"])
    .drop(
        columns=[
            "c.hosname",
            "c.provcode",
            "ch.changwatname",
            "c.distcode",
            "ca.ampurname",
            "c.subdistcode",
            "ct.tambonname",
        ]
    )
    .filter(
        (~pl.col("s.chiefcomp").str.contains("รับยา|ตามนัด|F/U|ต่อยา|มาตามนัด"))
        | (
            (pl.col("s.chiefcomp").str.contains("รับยา|ตามนัด|F/U|ต่อยา"))
            & pl.col("s.chiefcomp").str.contains("มีอาการ")
        )
    )
    .with_columns(
        zfill=pl.col("opd.hospcode")
        .cast(pl.String)
        .str.zfill(5)
        .alias("opd.hospcode")
        # .with_columns(
        #     opd_hospcode=pl.col("opd.hospcode").apply(lambda x: x.zfill(5) if isinstance(x, str) else str(x).zfill(5))
    )
    .join(hoscode, how="left", left_on="opd.hospcode", right_on="5_number")
)

C:\Users\brown\AppData\Local\Temp\ipykernel_14308\3608420559.py:2: DeprecationWarning: The argument `dtypes` for `read_csv` is deprecated. It has been renamed to `schema_overrides`.
  pl.read_csv(
C:\Users\brown\AppData\Local\Temp\ipykernel_14308\3608420559.py:18: DeprecationWarning: named `columns` param is deprecated; use positional `*args` instead.
  .drop(
C:\Users\brown\AppData\Local\Temp\ipykernel_14308\3608420559.py:44: DeprecationWarning: The default coalesce behavior of left join will change to `False` in the next breaking release. Pass `coalesce=True` to keep the current behavior and silence this warning.
  .join(hoscode, how="left", left_on="opd.hospcode", right_on="5_number")


In [9]:
data

opd.hospcode,opd.pid,opd.seq,p.cid,p.sex,p.birth,age,p.nation,p.typearea,opd.diagcode,opd.diagtype,opd.date_serv,p.occupation_new,occ.occupation_new,p.occupation_old,old.occupation,s.instype,s.typeout,s.chiefcomp,zfill,hosname,odpc,province,district,subdistrict
str,str,str,str,i64,str,i64,str,i64,str,str,str,str,str,i64,str,str,str,str,str,str,str,str,str,str
"""10857""","""347225""","""5316588""","""94afea6c32fb7eabb2414a7c478d05…",1,"""14/6/1967""",56,"""99""",4,"""I210""","""1""","""11/1/2024""","""9999""","""ไม่มีงานทำ""",0,"""ไม่มีอาชีพ""","""4200""","""1""","""#NAME?""","""10857""","""โรงพยาบาลกบินทร์บุรี""","""เขตสุขภาพที่ 6""","""ปราจีนบุรี""","""กบินทร์บุรี""","""กบินทร์"""
"""11394""","""590005607""","""1944928""","""f932a697b4dc378cc26213213183ed…",1,"""24/11/1959""",64,"""99""",4,"""I219""","""1""","""29/2/2024""","""9622""","""คนงานรับจ้างทั่วไป""",403,null,"""100""","""3""","""เจ็บหน้าอกมา 2 ชม. , #ญาตินำส่…","""11394""","""โรงพยาบาลรัตภูมิ""","""เขตสุขภาพที่ 12""","""สงขลา""","""รัตภูมิ""","""กำแพงเพชร"""
"""10705""","""77026""","""9107993""","""58c8512807fad2aaf8564097ed8706…",2,"""23/5/1967""",56,"""99""",4,"""I210""","""4""","""19/4/2024""","""6111""","""ผู้ปลูกพืชไร่และพืชผัก, ทำไร่,…",502,null,"""100""","""1""","""xสุกัญญา จำรูญศิริxมาผิดนัด 1…","""10705""","""โรงพยาบาลเลย""","""เขตสุขภาพที่ 8""","""เลย""","""เมืองเลย""","""กุดป่อง"""
"""11417""","""55433""","""1499986""","""158d8fb2501f961cc16d64ba8f4a3f…",1,"""1/8/1963""",60,"""99""",4,"""I219""","""1""","""23/4/2024""","""6111""","""ผู้ปลูกพืชไร่และพืชผัก, ทำไร่,…",null,null,"""8100""","""1""","""ติดตามเยี่ยมบ้านผู้ป่วยSTEMI พ…","""11417""","""โรงพยาบาลควนขนุน""",null,"""พัทลุง""","""ควนขนุน""","""ควนขนุน"""
"""12431""","""13484""","""135869TB""","""d9bd13d5407aa1e01491cf507fc223…",1,"""25/7/1947""",76,"""99""",1,"""I214""","""2""","""4/3/2024""","""5221""","""เจ้าของร้านค้า, ตัวแทนจำหน่ายห…",3,"""ค้าขาย""","""100""","""1""","""ญาติขอรับ set ทำแผล 2 setญาติ…","""12431""","""ศูนย์แพทย์ชุมชนคูหาสวรรค์""","""เขตสุขภาพที่ 12""","""พัทลุง""","""เมืองพัทลุง""","""คูหาสวรรค์"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""10670""","""65093232""","""R1130248""","""02abe478724df481ffd0b2ce52c722…",1,"""25/2/1957""",66,"""99""",4,"""I219""","""1""","""13/1/2024""",null,null,901,"""ไม่ระบุ""","""100""","""1""","""เจ็บหน้าอก 6 ชม""","""10670""","""โรงพยาบาลขอนแก่น""","""เขตสุขภาพที่ 7""","""ขอนแก่น""","""เมืองขอนแก่น""","""ในเมือง"""
"""10680""","""64939""","""9267428""","""fef42e40962200f7e86c88d3126c4a…",1,"""1/7/1952""",71,"""99""",4,"""I213""","""1""","""6/2/2024""","""9629""","""ผู้ประกอบวิชาชีพงานพื้นฐานอื่น…",900,"""หญิงบริการ""","""100""","""1""","""ติดตามการรักษา""","""10680""","""โรงพยาบาลมหาราชนครศรีธรรมราช""","""เขตสุขภาพที่ 11""","""นครศรีธรรมราช""","""เมืองนครศรีธรรมราช""","""ในเมือง"""
"""10680""","""143844""","""9232936""","""8ada8e05a3c8638e86bc175b19c83b…",2,"""14/4/1969""",54,"""99""",4,"""I213""","""1""","""23/1/2024""","""5221""","""เจ้าของร้านค้า, ตัวแทนจำหน่ายห…",606,null,"""100""","""1""","""มาก่อน ติดธุระ""","""10680""","""โรงพยาบาลมหาราชนครศรีธรรมราช""","""เขตสุขภาพที่ 11""","""นครศรีธรรมราช""","""เมืองนครศรีธรรมราช""","""ในเมือง"""


In [10]:
filtered_df = data.filter(data["opd.hospcode"] == "935")
filtered_df

opd.hospcode,opd.pid,opd.seq,p.cid,p.sex,p.birth,age,p.nation,p.typearea,opd.diagcode,opd.diagtype,opd.date_serv,p.occupation_new,occ.occupation_new,p.occupation_old,old.occupation,s.instype,s.typeout,s.chiefcomp,zfill,hosname,odpc,province,district,subdistrict
str,str,str,str,i64,str,i64,str,i64,str,str,str,str,str,i64,str,str,str,str,str,str,str,str,str,str
"""935""","""48568""","""3627858""","""df3ee7d44c5cef9f60d95db08f0898…",2,"""1/1/1936""",88,"""99""",2,"""I219""","""1""","""6/2/2024""","""5152""","""พนักงานดูแลงานบ้าน/แม่บ้าน""",14,"""งานบ้าน""","""100""","""1""","""ญาติขอใบส่งตัว""","""00935""",null,null,null,null,null
"""935""","""35681""","""3626808""","""828c26477d2b6173c20e13d2c6dfc7…",2,"""1/1/1950""",74,"""99""",1,"""I219""","""4""","""31/1/2024""","""5152""","""พนักงานดูแลงานบ้าน/แม่บ้าน""",14,"""งานบ้าน""","""100""","""1""","""ปวดบริเวณรักแร้ซ้าย เจ็บหน้าอก…","""00935""",null,null,null,null,null
"""935""","""22149""","""3630344""","""5e2178c1c6e87dcba3da09c972554b…",1,"""30/12/1955""",68,"""99""",1,"""I229""","""1""","""20/2/2024""","""9622""","""คนงานรับจ้างทั่วไป""",403,null,"""100""","""1""","""ขอใบส่งตัว""","""00935""",null,null,null,null,null
"""935""","""49511""","""3622940""","""ca8bac01cf09a653397be9820abd29…",1,"""1/1/1957""",67,"""99""",3,"""I249""","""4""","""3/1/2024""","""9622""","""คนงานรับจ้างทั่วไป""",403,null,"""100""","""1""","""ขอใบส่งตัว""","""00935""",null,null,null,null,null
"""935""","""66405""","""3623498""","""4a42b58b04a99bd3546ebc05cc55e6…",1,"""5/3/1939""",84,"""99""",1,"""I248""","""1""","""9/1/2024""","""9999""","""ไม่มีงานทำ""",0,"""ไม่มีอาชีพ""","""100""","""1""","""ญาติขอใบส่งตัวแทนผู้ป่วย""","""00935""",null,null,null,null,null


In [11]:
data = data.to_pandas()
data.loc[data["opd.hospcode"] == "99782", "name_th"] = (
    "ศูนย์สุขภาพชุมชนโรงพยาบาลสทิงพระ(สทิงพระ)"
)
data.loc[data["opd.hospcode"] == "99782", "province"] = "สงขลา"
data.loc[data["opd.hospcode"] == "99782", "district"] = "สทิงพระ"
data.loc[data["opd.hospcode"] == "99782", "subdistrict"] = "จะทิ้งพระ"

data.loc[data["opd.hospcode"] == "99942", "name_th"] = "ศูนย์สุขภาพชุมชนโรงพยาบาลบึงโขงหลง"
data.loc[data["opd.hospcode"] == "99942", "province"] = "บึงกาฬ"
data.loc[data["opd.hospcode"] == "99942", "district"] = "บึงโขงหลง"
data.loc[data["opd.hospcode"] == "99942", "subdistrict"] = "บึงโขงหลง"

data.loc[data["opd.hospcode"] == "99742", "name_th"] = (
    "หน่วยบริการปฐมภูมิ กลุ่มงานเวชปฏิบัติครอบครัวและชุมชน รพ.บันนังสตา"
)
data.loc[data["opd.hospcode"] == "99742", "province"] = "ยะลา"
data.loc[data["opd.hospcode"] == "99742", "district"] = "บันนังสตา"
data.loc[data["opd.hospcode"] == "99742", "subdistrict"] = "บันนังสตา"

data.loc[data["opd.hospcode"] == "99745", "name_th"] = (
    "ศูนย์สุขภาพชุมชนเมืองเวชกรรม (เมืองยะลา)"
)
data.loc[data["opd.hospcode"] == "99745", "province"] = "ยะลา"
data.loc[data["opd.hospcode"] == "99745", "district"] = "เมือง"
data.loc[data["opd.hospcode"] == "99745", "subdistrict"] = "สะเตง"

data.loc[data["opd.hospcode"] == "99763", "name_th"] = "ศูนย์สุขภาพชุมชนตำบลยามู"
data.loc[data["opd.hospcode"] == "99763", "province"] = "ปัตตานี"
data.loc[data["opd.hospcode"] == "99763", "district"] = "ยะหริ่ง"
data.loc[data["opd.hospcode"] == "99763", "subdistrict"] = "ยามู"


data.loc[data["name_th"].isnull(), "name_th"] = "ไม่ระบุ"
data.loc[data["name_th"].isnull(), "province"] = "ไม่ระบุ"
data.loc[data["name_th"].isnull(), "district"] = "ไม่ระบุ"
data.loc[data["name_th"].isnull(), "subdistrict"] = "ไม่ระบุ"

data.loc[data["opd.hospcode"] == "99874", "name_th"] = "ศุนย์สุขภาพชุมชนโรงพยาบาลยะรัง"
data.loc[data["opd.hospcode"] == "99874", "province"] = "ปัตตานี"
data.loc[data["opd.hospcode"] == "99874", "district"] = "ยะรัง"
data.loc[data["opd.hospcode"] == "99874", "subdistrict"] = "ปิตูมุดี"
data = pl.from_pandas(data)

In [12]:
data.describe()

statistic,opd.hospcode,opd.pid,opd.seq,p.cid,p.sex,p.birth,age,p.nation,p.typearea,opd.diagcode,opd.diagtype,opd.date_serv,p.occupation_new,occ.occupation_new,p.occupation_old,old.occupation,s.instype,s.typeout,s.chiefcomp,zfill,hosname,odpc,province,district,subdistrict,name_th
str,str,str,str,str,f64,str,f64,str,f64,str,str,str,str,str,f64,str,str,str,str,str,str,str,str,str,str,str
"""count""","""29318""","""29318""","""29318""","""29318""",29318.0,"""29316""",29318.0,"""29318""",29318.0,"""29318""","""29318""","""29318""","""28833""","""28703""",27717.0,"""6965""","""29318""","""29318""","""29318""","""29318""","""28791""","""25001""","""28791""","""28791""","""28791""","""29318"""
"""null_count""","""0""","""0""","""0""","""0""",0.0,"""2""",0.0,"""0""",0.0,"""0""","""0""","""0""","""485""","""615""",1601.0,"""22353""","""0""","""0""","""0""","""0""","""527""","""4317""","""527""","""527""","""527""","""0"""
"""mean""",null,null,null,null,1.379528,null,65.515417,null,3.670339,null,null,null,null,null,486.298625,null,null,null,null,null,null,null,null,null,null,null
"""std""",null,null,null,null,0.485278,null,20.599585,null,0.915632,null,null,null,null,null,271.899676,null,null,null,null,null,null,null,null,null,null,null
"""min""","""10025""","""000338-64""","""0013817BX""","""000022152ef5fc86a91edfb4c438a7…",1.0,"""0/1/1900""",0.0,"""0""",0.0,"""I21""","""0""","""1/1/2024""","""1""",""" ครูสอนศิลปะ""",0.0,"""ครู""","""0""","""0""","""# COPD admit 30/5-7/6/66 : COP…","""935""",""" โรงพยาบาลพุทธโสธร""","""เขตสุขภาพที่ 1""","""กระบี่""","""กงหรา""","""กบินทร์""","""ไม่ระบุ"""
"""25%""",null,null,null,null,1.0,null,57.0,null,4.0,null,null,null,null,null,403.0,null,null,null,null,null,null,null,null,null,null,null
"""50%""",null,null,null,null,1.0,null,66.0,null,4.0,null,null,null,null,null,503.0,null,null,null,null,null,null,null,null,null,null,null
"""75%""",null,null,null,null,2.0,null,74.0,null,4.0,null,null,null,null,null,606.0,null,null,null,null,null,null,null,null,null,null,null
"""max""","""99867""","""KS9949""","""R3290222""","""fffee8834eae3dfa0d86b413cf8c9a…",2.0,"""9/9/1995""",2021.0,"""999""",5.0,"""I249""","""7""","""9/5/2024""","""9999""","""ไม่มีงานทำ""",999.0,"""ไม่ระบุ""","""o1""","""9""","""�����������¹ 4 �������մ� 3 ��…","""99867""","""โรงพยาบาลไม้แก่น""","""เขตสุขภาพที่ 9""","""แม่ฮ่องสอน""","""ไม้แก่น""","""ไสหร้า""","""ไม่ระบุ"""


In [13]:
data_2 = (
    data.with_columns(
        pl.when(pl.col("occ.occupation_new").is_null())
        .then(pl.lit("ไม่ระบุ"))
        .otherwise(pl.col("occ.occupation_new"))
        .alias("occ.occupation_new")
    )
    .with_columns(
        pl.when(pl.col("p.sex") == 1)
        .then(pl.lit("ชาย"))
        .when(pl.col("p.sex") == 2)
        .then(pl.lit("หญิง"))
        .otherwise(pl.lit("ไม่ระบุ"))
        .alias("p.sex")
    )
    .with_columns(pl.col("opd.diagcode").str.replace("[.|-]", ""))
    .with_columns(pl.col("opd.diagcode").str.to_uppercase())
    .with_columns(
        pl.when(pl.col("opd.diagtype") == "1")
        .then(pl.lit("principal dx"))
        .when(pl.col("opd.diagtype") == "2")
        .then(pl.lit("co-morbidity"))
        .when(pl.col("opd.diagtype") == "3")
        .then(pl.lit("complication"))
        .when(pl.col("opd.diagtype") == "4")
        .then(pl.lit("other"))
        .when(pl.col("opd.diagtype") == "5")
        .then(pl.lit("external"))
        .when(pl.col("opd.diagtype") == "6")
        .then(pl.lit("additional"))
        .when(pl.col("opd.diagtype") == "0")
        .then(pl.lit("principal dx"))
        .otherwise(pl.lit("morph"))
        .alias("opd.diagtype")
    )
    .with_columns(dx_group=pl.lit("กลุ่มโรคหัวใจและหลอดเลือด"))
    .with_columns(
        pl.col("opd.date_serv")
        .str.to_date("%d/%m/%Y")
        .dt.strftime("%Y-%m-%d")
        .str.to_date()
        .alias("opd.date_serv")
        # pl.col("opd.date_serv")
        # .str.to_date("%Y-%m-%d")
        # .dt.strftime("%d-%m-%Y")
        # .alias("opd.date_serv")
    )
    .join(icd10, how="left", left_on="opd.diagcode", right_on="code")
    .join(nation, how="left", left_on="p.nation", right_on="code")
    .join(odpc, how="left", left_on="province", right_on="province")
    .group_by(["p.cid", "opd.date_serv"])
    .first()
    .select(
        [
            pl.col("opd.date_serv").alias("date_serv"),
            pl.col("odpc"),
            pl.col("province").alias("changwatname"),
            pl.col("district").alias("ampurname"),
            pl.col("subdistrict").alias("tambonname"),
            pl.col("opd.hospcode").alias("hospcode"),
            pl.col("name_th").alias("hosname"),
            pl.col("p.sex").alias("sex"),
            pl.col("p.birth").alias("birth"),
            pl.col("age"),
            pl.col("nation"),
            pl.col("s.chiefcomp").alias("chiefcomp"),
            pl.col("occ.occupation_new").alias("occupation"),
            pl.col("p.typearea").alias("typearea"),
            pl.col("opd.diagcode").alias("diagcode"),
            pl.col("opd.diagtype").alias("diagtype"),
            pl.col("dx_group").alias("typediag"),
            pl.col("dz_name").alias("diagname"),
        ]
    )
    .with_columns(
        pl.when(pl.col("nation").is_null())
        .then(pl.lit("ไม่ระบุ"))
        .otherwise(pl.col("nation"))
        .alias("nation")
    )
    .with_row_count(name="id", offset=1)
    .with_columns(create_date=datetime.date.today())
    .with_columns(
        pl.col("create_date").dt.strftime("%Y-%m-%d").str.to_date().alias("create_date")
    )
)

C:\Users\brown\AppData\Local\Temp\ipykernel_14308\184114954.py:48: DeprecationWarning: The default coalesce behavior of left join will change to `False` in the next breaking release. Pass `coalesce=True` to keep the current behavior and silence this warning.
  .join(icd10, how="left", left_on="opd.diagcode", right_on="code")
C:\Users\brown\AppData\Local\Temp\ipykernel_14308\184114954.py:49: DeprecationWarning: The default coalesce behavior of left join will change to `False` in the next breaking release. Pass `coalesce=True` to keep the current behavior and silence this warning.
  .join(nation, how="left", left_on="p.nation", right_on="code")
C:\Users\brown\AppData\Local\Temp\ipykernel_14308\184114954.py:50: DeprecationWarning: The default coalesce behavior of left join will change to `False` in the next breaking release. Pass `coalesce=True` to keep the current behavior and silence this warning.
  .join(odpc, how="left", left_on="province", right_on="province")
C:\Users\brown\AppData\

In [14]:
data_2.describe()

statistic,id,date_serv,odpc,changwatname,ampurname,tambonname,hospcode,hosname,sex,birth,age,nation,chiefcomp,occupation,typearea,diagcode,diagtype,typediag,diagname,create_date
str,f64,str,str,str,str,str,str,str,str,str,f64,str,str,str,f64,str,str,str,str,str
"""count""",27864.0,"""27864""","""23712""","""27364""","""27364""","""27364""","""27864""","""27864""","""27864""","""27863""",27864.0,"""27864""","""27864""","""27864""",27864.0,"""27864""","""27864""","""27864""","""27864""","""27864"""
"""null_count""",0.0,"""0""","""4152""","""500""","""500""","""500""","""0""","""0""","""0""","""1""",0.0,"""0""","""0""","""0""",0.0,"""0""","""0""","""0""","""0""","""0"""
"""mean""",13932.5,"""2024-02-26""",null,null,null,null,null,null,null,null,65.45374,null,null,null,3.671368,null,null,null,null,"""2024-07-12"""
"""std""",8043.788287,null,null,null,null,null,null,null,null,null,17.342399,null,null,null,0.914453,null,null,null,null,null
"""min""",1.0,"""2024-01-01""","""เขตสุขภาพที่ 1""","""กระบี่""","""กงหรา""","""กบินทร์""","""10025""","""ไม่ระบุ""","""ชาย""","""0/1/1900""",0.0,"""กะเหรี่ยง (ชุมชนบนพื้นที่สูง)""","""# COPD admit 30/5-7/6/66 : COP…",""" ครูสอนศิลปะ""",0.0,"""I21""","""co-morbidity""","""กลุ่มโรคหัวใจและหลอดเลือด""","""Acute coronary thrombosis not …","""2024-07-12"""
"""25%""",6967.0,"""2024-01-27""",null,null,null,null,null,null,null,null,57.0,null,null,null,4.0,null,null,null,null,"""2024-07-12"""
"""50%""",13933.0,"""2024-02-24""",null,null,null,null,null,null,null,null,66.0,null,null,null,4.0,null,null,null,null,"""2024-07-12"""
"""75%""",20898.0,"""2024-03-25""",null,null,null,null,null,null,null,null,74.0,null,null,null,4.0,null,null,null,null,"""2024-07-12"""
"""max""",27864.0,"""2024-05-20""","""เขตสุขภาพที่ 9""","""แม่ฮ่องสอน""","""ไม้แก่น""","""ไสหร้า""","""99867""","""ไม่ระบุ""","""หญิง""","""9/9/1995""",2021.0,"""ไม่ระบุ""","""�����������¹ 4 �������մ� 3 ��…","""ไม่ระบุ""",5.0,"""I249""","""principal dx""","""กลุ่มโรคหัวใจและหลอดเลือด""","""Subsequent STEMI of unsp site""","""2024-07-12"""


In [ ]:
data_2.write_csv("I21_I22_I24_01-052024.csv")

In [ ]:
# median_h10 = (
#     combined_h10.select(["opd.diagcode", "opd.date_serv", "province", "district"])
#     .drop_nulls(subset=["province", "district"])
#     .with_columns(
#         week=pl.col("opd.date_serv").str.to_date("%Y-%m-%d").dt.strftime("%U")
#     )
#     .with_columns(
#         year=pl.col("opd.date_serv").str.to_date("%Y-%m-%d").dt.strftime("%Y")
#     )
#     .with_columns(
#         pl.when(pl.col("week") == "00")
#         .then(pl.lit("52"))
#         .otherwise(pl.col("week"))
#         .alias("week")
#     )
#     .drop(columns="opd.date_serv")
#     .group_by(["year", "week", "province", "district"], maintain_order=True)
#     .agg(pl.col("opd.diagcode").alias("no_patients").count())
#     .join(
#         total_pop,
#         how="left",
#         left_on=["province", "district", "year"],
#         right_on=["ProvinceThai", "DistrictThaiShort", "year"],
#     )
#     .filter(~pl.col("total_pop").is_null())
#     .with_columns(
#         rate_per_100000=(pl.col("no_patients") / pl.col("total_pop") * 100000)
#     )
#     .drop(columns=["year", "no_patients", "total_pop"])
#     .group_by(by=["week", "province", "district"])
#     .agg(pl.col("rate_per_100000").median().alias("median_rate_per_100000"))
# )